**Leer antes de iniciar**

Esta informacion es parte de la Encuesta sobre la Comunidad Estadounidense (ACS, por sus siglas en inglés), una encuesta continua realizada por la Oficina del Censo de los EE.UU. que recopila información detallada demográfica, social, económica y de vivienda de una muestra representativa de hogares en todo el país para proporcionar datos vitales anualmente.


Esta información es del último año disponible públicamente, **2022**. Fue accedida a través del sistema PUMS (Sistema de Microdatos de Uso Público) de la Oficina del Censo de los EE.UU.
https://data.census.gov/


Toda la información aquí está a nivel de **PUMA**. PUMA, Área de Microdatos de Uso Público, es una unidad geográfica utilizada en los microdatos del Censo de EE.UU. que representa un área con una población de aproximadamente 100,000 personas, lo que permite la publicación de datos demográficos y socioeconómicos detallados mientras se protege la privacidad de los individuos.


Debido a que la densidad de población varía entre los condados, algunos condados muy poblados contienen múltiples PUMA. En otros casos, un PUMA podría contener varios condados muy escasamente poblados. Lo importante es que cada PUMA contiene aproximadamente 100,000-150,000 personas.


Hay 168 PUMA en el estado de Florida.

Después de algunos intentos y errores, me di cuenta de que era más fácil exportar archivos CSV para cada categoría de datos que queríamos analizar y combinarlos con Python, en lugar de intentar combinarlos con la plataforma en línea de la Oficina del Censo.


Así que, este ETL:
- combinará cuatro archivos CSV en un solo DataFrame
- asignará un geocódigo a cada región PUMAS para su uso posterior
- probará si hay valores nulos y duplicados
- creará nuevo archivo csv para consumo general

________________________________________________________________________

*Importación de librerias*

In [ ]:
import pandas as pd

*Abrir archivos csv y crear dataframes*

In [ ]:
csv_path = r'census_pop\age.csv'

df_age = pd.read_csv(csv_path, skiprows=3)

df_age

In [ ]:
csv_path = r'census_pop\income.csv'

df_income = pd.read_csv(csv_path, skiprows=3)

df_income

In [ ]:
csv_path = r'census_pop\avg_prop_val.csv'

df_avg_prop_val = pd.read_csv(csv_path, skiprows=3)

df_avg_prop_val

*Mergear tres dataframes en uno*

In [ ]:
df_1 = pd.read_csv(r'census_pop\age.csv', skiprows=3)
df_2 = pd.read_csv(r'census_pop\avg_prop_val.csv', skiprows=3)
df_3 = pd.read_csv(r'census_pop\income.csv', skiprows=3)


df_merge_1_2 = pd.merge(df_1, df_2, on='Selected Geographies')

df_merge = pd.merge(df_merge_1_2, df_3, on='Selected Geographies')

print(df_merge.head())

*Añadir un geocódigo simple a cada área PUMA*

Usando un archivo de geoshapes, podemos añadir este geocódigo PUMA a los negocios en nuestros otros conjuntos de datos (Google/Yelp) utilizando sus coordenadas de latitud/longitud. Esto sería el key que conecta esta información con esos conjuntos de datos si quisiéramos hacerlo.

Del mismo modo, podría usarse en la fase de visualización (un inversor busca un negocio, encuentra negocios similares con reseñas similares, pero también información sobre impuestos a la propiedad promedio, ingreso familiar promedio, edad promedio de la poblacion, valor promedio de la propiedad, etc para esa área).

In [ ]:
unique_geographies = df_merge['Selected Geographies'].unique()
geo_mapping = {geo: code for code, geo in enumerate(unique_geographies, start=1)}

df_merge['Geography_Code'] = df_merge['Selected Geographies'].map(geo_mapping)

In [ ]:
cols = df_merge.columns.tolist()
cols.insert(1, cols.pop(cols.index('Geography_Code')))
df_merge_geo = df_merge[cols]

*He decidido añadir otro DataFrame porque estos archivos son ligeros, así que...por qué no?*

In [ ]:
df_tax = pd.read_csv(r'census_pop\avg_prop_tax.csv', skiprows=3)

df_tax

In [ ]:
df_merge_new = pd.merge(df_merge_geo, df_tax, on='Selected Geographies')

*Nulos, duplicados, valores faltantes*

In [ ]:
null_values = df_merge_new.isnull().sum()
print("Null values in each column:")
print(null_values)

duplicates = df_merge_new.duplicated().sum()
print("\nNumber of duplicate rows:")
print(duplicates)

missing_values = df_merge_new.isnull().sum().sum()
print("\nTotal number of missing values in the DataFrame:")
print(missing_values)

columns_with_missing = df_merge_new.columns[df_merge_new.isnull().any()].tolist()
print("\nColumns with missing values:")
print(columns_with_missing)

*Convertir nombres de columnas*

In [ ]:
df_final = df_merge_new.rename(columns={
    'Selected Geographies': 'GEO',
    'Geography_Code': 'GEO_CODE',
    'Age': 'Age (avg)',
    'Property value': 'Property value (avg)',
    'Household income (past 12 months, use ADJINC to adjust HINCP to constant dollars)': 'Household income (avg)',
    'Property taxes (yearly real estate taxes)': 'Property taxes (avg)'
})

*Convertir dataset limpio a csv*

In [ ]:
df_final.to_csv('dataset_censo.csv', index=False)

Diccionario:

*   'GEO': Area geografica (PUMA)

*   'GEO_CODE': Codigo numerico de PUMA

*   'Age (avg)': Edad promedia de la area PUMA

*   'Property value (avg)': Valor de propiedad promedio dentro de la area PUMA

*   'Household income (avg)': ingreso promedio anual por hogar dentro de la area PUMA

*  'Property taxes (avg)': impuesto promedio de propiedad dentro de la area PUMA

Tengo más conjuntos de datos: etnicidad, división de género, estado laboral, nivel educativo, etc., que podrían añadirse fácilmente.

Es una cuestión de decidir qué consideramos útil para ML o Visualización a medida que este proyecto se desarrolla.